In [4]:
#Install Packages

!pip install faiss-cpu
!pip install sentence-transformers

   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
    --------------------------------------- 0.3/13.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.8 MB 1.9 MB/s eta 0:00:08
   --- ------------------------------------ 1.0/13.8 MB 1.9 MB/s eta 0:00:07
   --- ------------------------------------ 1.3/13.8 MB 1.9 MB/s eta 0:00:07
   ---- ----------------------------------- 1.6/13.8 MB 1.7 MB/s eta 0:00:08
   ----- ---------------------------------- 1.8/13.8 MB 1.6 MB/s eta 0:00:08
   ------ --------------------------------- 2.1/13.8 MB 1.5 MB/s eta 0:00:08
   ------ --------------------------------- 2.1/13.8 MB 1.5 MB/s eta 0:00:08
   ------ --------------------------------- 2.4/13.8 MB 1.3 MB/s eta 0:00:09
   ------- -------------------------------- 2.6/13.8 MB 1.3 MB/s eta 0:00:09
   -------- ------------------------------- 2.9/13.8 MB 1.3 MB/s eta 0:00:09
   --------- ------------------------------ 3.4/13.8 MB 1.4 MB/s eta 0:00:08
   ----------

In [8]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [10]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [11]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [12]:
pip install sentence-transformers

### Step 1 : Create source embeddings for the text column

In [13]:
from sentence_transformers import SentenceTransformer

In [14]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [15]:
vectors.shape

(8, 768)

In [16]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [2]:
conda install -c conda-forge faiss-cpu

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: D:\Anaconda

  added / updated specs:
    - faiss-cpu


The following NEW packages will be INSTALLED:

  faiss              conda-forge/win-64::faiss-1.8.0-py312hd867dbf_1_cpu 
  faiss-cpu          conda-forge/win-64::faiss-cpu-1.8.0-hf9c7e24_1 
  libblas            conda-forge/win-64::libblas-3.9.0-1_h8933c1f_netlib 
  libexpat           conda-forge/win-64::libexpat-2.6.3-he0c23c2_0 
  libfaiss           conda-forge/win-64::libfaiss-1.8.0-hb15a273_1_cpu 
  liblapack          conda-forge/win-64::liblapack-3.9.0-8_h719fc58_netlib 
  libsqlite          conda-forge/win-64::libsqlite-3.47.0-h2466b09_1 
  libzlib            conda-forge/win-64::libzlib-1.2.13-h2466b09_6 
  m2w64-gcc-libgfor~ conda-forge/win-64::m2w64-gcc-libgfortran-5.3.0-6 
  m2w64-gcc-libs     conda-forge/win-64::m2w64-gcc-libs-5.3.0-7 
  m2w64-gcc-libs-co~ conda-forge/win-64::m2w64-gcc

In [17]:
import faiss

index = faiss.IndexFlatL2(dim)


### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [18]:
index.add(vectors)

In [19]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000021F0B145590> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [20]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [23]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [24]:
index.search(svec,k=2)

(array([[1.3844836, 1.4039096]], dtype=float32), array([[3, 2]], dtype=int64))

In [ ]:
# faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [25]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844836, 1.4039096]], dtype=float32)

In [26]:
I

array([[3, 2]], dtype=int64)

In [27]:
I.tolist()

[[3, 2]]

In [28]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [29]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [30]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query